<a href="https://colab.research.google.com/github/jtghchau/TextClassification/blob/main/pytorchProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#Loading MNIST
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

trainData = torchvision.datasets.MNIST(root='data', train=True, download=True, transform=transform)
testData = torchvision.datasets.MNIST(root='data', train=False, download=True, transform=transform)

trainLoader = DataLoader(trainData, batch_size=64, shuffle=True)
testLoader = DataLoader(testData, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 15.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 495kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.39MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.69MB/s]


In [4]:
#Building the multi-layer perceptron
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, output_size=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
#Training the MLP

!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/mlp_experiment')
#!tensorboard --logdir==runs/mlp_experiment

mlpModel = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlpModel.parameters(), lr=0.001)
writer = SummaryWriter("logs/mlp")

for epoch in range(10):
    mlpModel.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in trainLoader:
        optimizer.zero_grad()
        outputs = mlpModel(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss = total_loss / len(trainLoader)

    mlpModel.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testLoader:
            outputs = mlpModel(images)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    test_acc = correct / total

    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Acc/test", test_acc, epoch)
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Test Acc={test_acc:.4f}")


Epoch 1: Train Loss=0.2482, Train Acc=0.9267, Test Acc=0.9626
Epoch 2: Train Loss=0.1035, Train Acc=0.9685, Test Acc=0.9711
Epoch 3: Train Loss=0.0733, Train Acc=0.9776, Test Acc=0.9749
Epoch 4: Train Loss=0.0567, Train Acc=0.9822, Test Acc=0.9751
Epoch 5: Train Loss=0.0435, Train Acc=0.9859, Test Acc=0.9719
Epoch 6: Train Loss=0.0382, Train Acc=0.9873, Test Acc=0.9764
Epoch 7: Train Loss=0.0332, Train Acc=0.9892, Test Acc=0.9778
Epoch 8: Train Loss=0.0263, Train Acc=0.9911, Test Acc=0.9786
Epoch 9: Train Loss=0.0257, Train Acc=0.9913, Test Acc=0.9801
Epoch 10: Train Loss=0.0235, Train Acc=0.9921, Test Acc=0.9762


/bin/bash: line 1: cd: runs/: No such file or directory
